# Homework 2: Multiclass Classification with PyTorch

In this assignment, you will build, train, and evaluate a neural network for multiclass classification using PyTorch.
You will use the [Garbage dataset](https://www.kaggle.com/datasets/mostafaabla/garbage-classification).
The goal is to gain hands-on experience with:
- Dataset preparation 
- Building two  PyTorch models
- Loss functions for multiclass
- Training loop and evaluation
- Visualize of performance

## About Dataset
### Context
This dataset has 15,150 images from 12 different classes of household garbage; paper, cardboard, biological, metal, plastic, green-glass, brown-glass, white-glass, clothes, shoes, batteries, and trash.

Garbage Recycling is a key aspect of preserving our environment. To make the recycling process possible/easier, the garbage must be sorted to groups that have similar recycling process. I found that most available data sets classify garbage into a few classes (2 to 6 classes at most). Having the ability to sort the household garbage into more classes can result in dramatically increasing the percentage of the recycled garbage.

### Content
An ideal setting for data collection would be to place a camera above a conveyor where the garbage comes one by one, so that the camera can capture real garbage images. But since such a setup is not feasible at the moment I collected most of the images in this dataset by web scraping, I tried to get images close to garbage images whenever possible, for example in biological garbage category I searched for rotten vegetables, rotten fruits and food remains, etc. However, for some classes such as clothes or shoes it was more difficult to get images of clothes or shoes from the garbage, so mostly it was images of normal clothes. Nevertheless, being able to classify the images of this data set to 12 classes can be a big step towards improving the recycling process.

### Imports

In [ ]:
import torch
import torchvision
import torch.nn as nn
import kagglehub
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import os
import torch.nn.functional as F
from tqdm import tqdm
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms


torch.manual_seed(42)


### Dwonload and prepare dataset from kagglehub
`kagglehub.dataset_download` downloads and extracts Kaggle datasets to a local cache directory (usually under `~/.cache/kagglehub/datasets/`). It returns the path to the unzipped dataset, preserving the original folder structure as found on Kaggle, such as one subfolder per class for image datasets.

---

### **What is the structure of the downloaded content?**

* Inside the returned directory (`path`), you will find the files and folders as originally organized on Kaggle.
* For the **garbage classification** dataset, you typically get a folder like:

  ```
  garbage_classification/
      cardboard/
      glass/
      metal/
      paper/
      plastic/
      trash/
      ...
  ```

  Each subfolder contains images belonging to that class (a classic structure for use with `torchvision.datasets.ImageFolder`).


In [3]:
# Download the latest version of the Kaggle dataset to a local directory
path = kagglehub.dataset_download("mostafaabla/garbage-classification",)

# Set the data directory to the location of the downloaded images
data_dir = os.path.join(path, "garbage_classification")

# Set the desired image size for resizing
img_size = 64

# Define the image transformations to apply to each image:
# - Resize the image to (img_size, img_size)
# - Convert the image to a PyTorch tensor
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),  # Resize images
    transforms.ToTensor()                     # Convert to tensor
])  

print(f"image transform: {transform}")

image transform: Compose(
    Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)


In [ ]:
dataset  = ImageFolder(root=data_dir, transform=transform)

classes_names = dataset.classes

# dataset = ImageFolder(data_dir + '/Training', transform=transform)
print("Number of images:", len(dataset), "number of classes:", len(classes_names))
print("Class names:", classes_names)

#fig, axes = plt.subplots(4, 3)


C:\Users\lpardess\.cache\kagglehub\datasets\mostafaabla\garbage-classification\versions\1\garbage_classification
Number of images: 15515 number of classes: 12
Class names: ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']


### TODO 1:
Create a 4×3 subplot that displays one example image from each category in the dataset.



In [5]:
# your coimport matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

def show_images(dataset):
    fig, axes = plt.subplots(4, 3)
    axes = axes.flatten() 

    shown = set()

    for idx, class_name in enumerate(dataset.classes):
        for img, label in dataset: # img is of size torch.Size([3, 64, 64])
            if dataset.classes[label] == class_name and class_name not in shown:
                axes[idx].imshow(np.transpose(img.numpy(), (1, 2, 0)))
                axes[idx].set_title(class_name)
                axes[idx].axis('off')
                shown.add(class_name)
                break

    plt.tight_layout()
    plt.show()

#TODO 1 delete
def show_image(dataset, index):
    fig, axes = plt.subplots(1, 1)

    img, lable = dataset[index];
    axes.imshow(np.transpose(img.numpy(), (1, 2, 0)))
    axes.axis('off')
    plt.tight_layout()
    plt.show()


### TODO 2:
Shuffle the dataset and split it into training and validation sets, using 80% of the samples for training and 20% for validation. Make sure that the class distribution is preserved as much as possible in both splits.


In [6]:
from torch.utils.data import Subset

def spit_dataset(dataset):
    dataset_size = len(dataset) 
    # Create train/test split (80/20)
    train_size = int(0.8 * dataset_size)
    test_size = len(dataset) - train_size
    indices = torch.randperm(dataset_size)
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    train_dataset = Subset(dataset, train_indices)
    test_dataset = Subset(dataset, test_indices)
    return train_dataset, test_dataset

def validate_distribution(dataset, title):
    #TODO 1 imp
    return;

train_dataset, test_dataset = spit_dataset(dataset)


### TODO 3:
Visualize the class distribution in both the training and validation sets using a bar plot, so you can compare how well the splits represent the overall dataset.


In [7]:
# your code here
def show_distribution(subset, set_name):
    print("Distuction of", set_name, ": ")
    class_sizes = [0] * len(classes_names)
    for img, label in subset:
        class_sizes[label] += 1

    for idx in range(len(classes_names)):
        percent = 100 * class_sizes[idx] / len(subset)
        print(f"Class '{classes_names[idx]}': {class_sizes[idx]} images ({percent:.2f}%)")

def get_class_counts(subset):
    class_sizes = [0] * len(classes_names)
    for img, label in subset:
        class_sizes[label] += 1
    return class_sizes

def plot_class_distribution(train_dataset, test_dataset):
    num_classes = len(classes_names)
    train_counts = get_class_counts(train_dataset)
    val_counts = get_class_counts(test_dataset)

    # Convert counts to relative frequencies
    train_total = sum(train_counts)
    val_total = sum(val_counts)
    train_rel = [count / train_total for count in train_counts]
    val_rel = [count / val_total for count in val_counts]

    x = range(num_classes)
    width = 0.35  # width of the bars

    plt.figure(figsize=(12, 6))
    plt.bar(x, train_rel, width=width, label='Train')
    plt.bar([i + width for i in x], val_rel, width=width, label='Validation')
    plt.xticks([i + width/2 for i in x], classes_names, rotation=45)
    plt.xlabel('Class')
    plt.ylabel('Relative amount')
    plt.title('Relative Class Distribution in Train and Validation Sets')
    plt.legend()
    plt.tight_layout()
    plt.show()

#plot_class_distribution(train_dataset=train_dataset, test_dataset = test_dataset)

### TODO 4:
Ensure that no single category accounts for more than 15% of the samples in the training set. If necessary, downsample the dominant classes. Then, visualize the new class distribution in the training set using a bar plot.


In [8]:
# your code here
import random

max_clothes = int(0.15 * len(train_dataset))
clothes_class_idx = classes_names.index("clothes")
clothes_indices = [i for i, idx in enumerate(train_dataset.indices)
                   if dataset.samples[idx][1] == clothes_class_idx]

random.seed(42) 
selected_clothes_indices = random.sample(clothes_indices, max_clothes)
other_indices = [i for i in range(len(train_dataset.indices)) if i not in clothes_indices]
final_indices = [train_dataset.indices[i] for i in other_indices + selected_clothes_indices]

train_dataset = Subset(dataset, final_indices)

#plot_class_distribution(train_dataset=train_dataset, test_dataset = test_dataset)

## Implementation of Regularization Layers

Implement two regularization layers from scratch:
1. `BatchNorm2d`
2. `LayerNorm`

Make sure that all trainable parameters (such as scale and shift) are properly registered as part of the computational graph, so they are optimized during training.


### TODO 5:
Implement the BatchNorm2d layer from scratch using only basic PyTorch components (such as `nn.Module` and tensor operations), without relying on `nn.BatchNorm2d`.


In [9]:
import torch
import torch.nn as nn

class MyBatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.num_features = num_features  # Number of feature channels (C in NCHW)
        self.eps = eps  # Small value to avoid division by zero when normalizing
        self.momentum = momentum  # Controls how quickly running stats are updated

        # Learnable scale (gamma) and shift (beta) parameters, one per channel.
        # These allow the network to undo normalization if needed, preserving representational power.
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

        # Running mean and variance are used during inference (model.eval()).
        # They accumulate a moving average of the mean/variance seen during training.
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        # x shape: (N, C, H, W) - batch, channels, height, width

        if self.training:
            # During training, compute mean and variance for each channel over the batch and spatial dims.
            # This normalizes each feature map (channel) so that its output has mean 0 and variance 1.
            mean = x.mean(dim=(0, 2, 3), keepdim=True)  # Shape: (1, C, 1, 1)
            var = x.var(dim=(0, 2, 3), unbiased=False, keepdim=True)  # Shape: (1, C, 1, 1)

            # Update running statistics for use during inference.
            # This helps the model make consistent predictions on unseen data (test/production).
            # Detach is used so that these updates are not part of the gradient computation.
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean.view(-1).detach()
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var.view(-1).detach()
        else:
            # During inference, use the running mean/var instead of batch statistics.
            # This ensures deterministic output and avoids dependence on batch size.
            mean = self.running_mean.view(1, -1, 1, 1)
            var = self.running_var.view(1, -1, 1, 1)

        # Normalize: subtract mean and divide by stddev (add eps for stability).
        # This reduces internal covariate shift, making training faster and more stable.
        x_hat = (x - mean) / torch.sqrt(var + self.eps)

        # Scale and shift: allow the network to learn optimal mean/variance for each channel.
        # This step ensures the transformation can represent the identity function if needed.
        out = self.gamma.view(1, -1, 1, 1) * x_hat + self.beta.view(1, -1, 1, 1)

        # The output is normalized, but can be rescaled and shifted as needed for learning.
        return out

### TODO 6:
Implement the LayerNorm layer from scratch using only basic PyTorch components (such as `nn.Module` and tensor operations), without relying on `nn.LayerNorm`.


In [10]:
# your code here

import torch
import torch.nn as nn

class MyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        """
        Args:
            normalized_shape: input shape from an expected input of size
                [*, normalized_shape[0], normalized_shape[1], ..., normalized_shape[-1]]
                Typically, for images, this is the number of channels (C), height (H), and width (W).
            eps: a value added to the denominator for numerical stability.
        """
        super().__init__()
        # Store the shape over which normalization is performed
        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)
        self.normalized_shape = tuple(normalized_shape)
        self.eps = eps

        # Learnable parameters for scaling (gamma) and shifting (beta)
        # These allow the network to undo normalization if needed
        self.gamma = nn.Parameter(torch.ones(self.normalized_shape))
        self.beta = nn.Parameter(torch.zeros(self.normalized_shape))

    def forward(self, x):
        # Compute mean and variance over the last N dimensions (those in normalized_shape)
        # This is different from BatchNorm, which normalizes over batch and spatial dims
        dims = tuple(-i for i in range(1, len(self.normalized_shape)+1))
        mean = x.mean(dim=dims, keepdim=True)  # Mean over normalized_shape
        var = x.var(dim=dims, unbiased=False, keepdim=True)  # Variance over normalized_shape

        # Normalize: subtract mean and divide by stddev (add eps for stability)
        # This ensures each sample (not batch) has zero mean and unit variance over the normalized_shape
        x_hat = (x - mean) / torch.sqrt(var + self.eps)

        # Scale and shift: apply learnable gamma and beta
        # Broadcasting ensures gamma and beta are applied to the correct dimensions
        out = self.gamma * x_hat + self.beta

        # Output is normalized per sample, not per batch, which helps with stability
        # especially in recurrent networks or when batch sizes are small
        return out

## Traning 

### TODO 7:
Complete the `GarbageClassifier` neural network by designing and implementing an architecture of your choice.  
Make use of the provided `_block` and `_block_mp` building blocks as you see fit.  
Allow the regularization type (e.g., `BatchNorm2d` or `LayerNorm`) to be specified from outside the class, so you can later compare the results between the two types of regularization.


In [11]:
class GarbageClassifier(nn.Module):
    def __init__(self, num_classes, norm_layer):
        # Fix: Remove norm_layer from super() call
        super(GarbageClassifier, self).__init__()
        
        # Set default normalization layer if none provided
        if norm_layer is None:
            norm_layer = MyBatchNorm2d 

        self.features = nn.Sequential(
            self._block_mp(3, 96, kernel_size=11, stride=4, padding=0, norm_layer=norm_layer),
            self._block_mp(96, 256, kernel_size=5, stride=1, padding=2, norm_layer=norm_layer),
            self._block(256, 384, kernel_size=3, stride=1, padding=1, norm_layer=norm_layer),
            self._block(384, 384, kernel_size=3, stride=1, padding=1, norm_layer=norm_layer),
            self._block_mp(384, 256, kernel_size=3, stride=1, padding=1, norm_layer=norm_layer),
        )   
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

       
    def _block_mp(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, norm_layer=None, kernel_size_mp=2):
        """AlexNet style block with max pooling"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            norm_layer(out_channels), # Normalization layer that you have implemented
            nn.MaxPool2d(kernel_size=kernel_size_mp),
            nn.ReLU(inplace=True)
        )

    def _block(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, norm_layer=None):
        """AlexNet style block without max pooling"""

        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            norm_layer(out_channels),
            nn.ReLU(inplace=True)
        )
    

    def forward(self, x):
        # Forward pass through the feature extractor givven self.features and self.classifier
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

### TODO 8:
Prepare all components needed for training:
1. Build your neural network with one type of regularization.
2. Create DataLoaders for the training (and optionally validation) sets.
3. Define the loss criterion.
4. Define the optimizer and assign it the trainable parameters of your model.
5. Print a summary of your network architecture.


In [ ]:
# your conde here
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

modelBatchNorm = GarbageClassifier(num_classes=len(classes_names), norm_layer=MyBatchNorm2d)
modelBatchNorm = modelBatchNorm.to(device)
modelLayerNorm = GarbageClassifier(num_classes=len(classes_names), norm_layer=MyLayerNorm)
modelLayerNorm = modelLayerNorm.to(device) 

lr = 0.0002
batch_size = 128
epoches = 10
optimizerBatchNorm = torch.optim.Adam(modelBatchNorm.parameters(), lr=lr)
optimizerLayerNorm = torch.optim.Adam(modelLayerNorm.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

12.1


### TODO 9:
Write a training loop to train your network for 10 epochs using the training set.
- Track and print the training loss for each epoch.
- After each epoch, compute and store both the loss and accuracy on the test set.
- After training, plot both the training and test losses on the same graph to visualize the learning process.
- Your model should achieve at least 75% accuracy on the test set.
- Remember to set your model to training mode (`model.train()`) during training, and to evaluation mode (`model.eval()`) when computing metrics on the test set.


In [13]:
# your code
def train_model(model, train_loader, test_loader, loss_fn, optimizer, device, epoches, classes_names):
    for epoch in range(epoches):
        model.train()  # Set the model to training mode
        running_loss = 0.0  # Track the loss for each epoch

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epoches}"):
            images, labels = images.to(device), labels.to(device)  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradient buffers
            outputs = model(images)  # Forward pass
            loss = loss_fn(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update model weights

            running_loss += loss.item()  # Accumulate loss

        # Print loss for the epoch
        print(f"Epoch [{epoch+1}/{epoches}], Loss: {running_loss/len(train_loader):.4f}")


        import torch


def test_model(model, test_loader, device, classes_names):
        # Set model to evaluation mode
    model.eval()

    # Initialize counters
    class_correct = [0 for _ in range(len(classes_names))]
    class_total = [0 for _ in range(len(classes_names))]

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            # Compare predictions to true labels
            for i in range(len(labels)):
                label = labels[i].item()
                pred = predicted[i].item()
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1


    print("Per-class accuracy:")
    for i, class_name in enumerate(classes_names):
        if class_total[i] > 0:
            accuracy = 100 * class_correct[i] / class_total[i]
            print(f"Class {class_name}: {accuracy:.2f}%")
        else:
            print(f"Class {class_name}: No samples in test set.")

train_model(modelBatchNorm, train_loader, test_loader, loss_fn, optimizerBatchNorm, device, epoches, classes_names)
test_model(modelBatchNorm, test_loader, device, classes_names)

Epoch 1/10: 100%|██████████| 79/79 [02:16<00:00,  1.73s/it]


Epoch [1/10], Loss: 1.6173


Epoch 2/10: 100%|██████████| 79/79 [00:33<00:00,  2.34it/s]


Epoch [2/10], Loss: 1.2216


Epoch 3/10: 100%|██████████| 79/79 [00:32<00:00,  2.43it/s]


...

Epoch [8/10], Loss: 0.5768


Epoch 9/10: 100%|██████████| 79/79 [00:36<00:00,  2.14it/s]


Epoch [9/10], Loss: 0.4943


Epoch 10/10: 100%|██████████| 79/79 [00:39<00:00,  2.01it/s]


Epoch [10/10], Loss: 0.4509
Per-class accuracy:
Class battery: 58.85%
Class biological: 69.83%
Class brown-glass: 83.97%
Class cardboard: 75.47%
Class clothes: 80.57%
Class green-glass: 69.23%
Class metal: 27.40%
Class paper: 72.81%
Class plastic: 21.38%
Class shoes: 80.05%
Class trash: 59.71%
Class white-glass: 45.14%


In [14]:
#test_model(modelBatchNorm, test_loader, device, classes_names)

#train_model(modelBatchNorm, train_loader, test_loader, loss_fn, optimizerBatchNorm, device, epoches, classes_names)
#test_model(modelBatchNorm, test_loader, device, classes_names)

### TODO 10:
Compute and report the accuracy of your trained model on the test set for each individual category (class).

For example:
* Class battery: 80%
* Class biological: 71%
* Class brown-glass: 70%
* Class cardboard: 85%
* Class clothes: 92%
* Class green-glass: 88%
* Class metal: 43%
* Class paper: 54%
* Class plastic: 39%
* Class shoes: 71%
* Class trash: 68%
* Class white-glass: 55%


### Repeat TODOs 8,9 and 10 with the other type of regularization

Repeat the previous steps for preparing your model, DataLoaders, optimizer, and training loop, but this time using the alternative regularization layer. After training, compare the performance between the two regularization types.


In [15]:
#test_model(modelBatchNorm, test_loader, device, classes_names)

#train_model(modelLayerNorm, train_loader, test_loader, loss_fn, optimizerLayerNorm, device, epoches, classes_names)
#test_model(modelLayerNorm, test_loader, device, classes_names)

### Final Task: Summary and Analysis

Write a summary of your work and the results you obtained. In 3–4 paragraphs, discuss your approach, key findings, and any challenges you encountered. Compare the performance of the two different regularization techniques you implemented, and suggest possible reasons for any differences you observed. Reflect on what you learned and what you might try differently in future experiments.


Good luck!